In [ ]:
%pip install pandas
%pip install folium

In [ ]:
import folium
import pandas as pd
import io
import matplotlib.pyplot as plt


# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster


In [ ]:
#import burden/prevalence data
data = pd.read_csv('./data.csv')
data.head()

#import lat/long for sites
sites = pd.read_csv('./sites.csv')

sites.head()

#merge the two datasets
full = pd.merge(data, sites)

full.head()

In [ ]:
##get mean prev and burden by site at each timepoint, create empty variable
full.reset_index()
full_group = full.groupby(['site','date','lat','long','NAME','Label'], as_index=False).agg(mean_burden=('barr burden','mean'),infected=('barr prev','mean'))
full_group['uninfected'] = 1 - full_group['infected']
full_group['empty'] = 0

##create rows for missing sample periods within sites. marked as empty
for index, site in sites.iterrows():
    for date in range(1,10):
        row = full_group.loc[(full_group['site'] == site.site) & (full_group['date'] == date)]
        if row.empty:
            full_group = full_group._append(
                {
                    "site":site.site,
                    "date":date,
                    "lat":site.lat,
                    "long":site.long,
                    "NAME":site.NAME,
                    "Label":site.Label,
                    "mean_burden":0,
                    "infected":0,
                    "uninfected":0,
                    "empty":1
                 },
                 ignore_index=True
            )

full_group.reset_index()
full_group.sort_values(by=['site', 'date'], inplace=True, ignore_index=True)
full_group.reset_index()

full_group



In [ ]:
##create pie charts
pie_charts_data = zip(full_group["infected"], full_group["uninfected"], full_group["empty"])
fig = plt.figure(figsize=(0.5, 0.5))
fig.patch.set_alpha(0)
ax = fig.add_subplot(111)
plots = []
for sizes in pie_charts_data:
    ax.pie(sizes, colors=("#e6194b", "#19e6b4", "#cccccc"))
    buff = io.StringIO()
    plt.savefig(buff, format="SVG")
    buff.seek(0)
    svg = buff.read()
    svg = svg.replace("\n", "")
    plots.append(svg)
    plt.cla()
plt.clf()
plt.close()




In [ ]:
# Start location for map
start_coordinate = [40.4600201,-89.0124789]
map = folium.Map(location=start_coordinate, zoom_start=9)

In [ ]:

#create clusters of piecharts 
marker_cluster = MarkerCluster()
map.add_child(marker_cluster)
i = 0
for lat, long in zip(full_group.lat, full_group.long):
    marker = folium.Marker([lat, long])
    icon = folium.DivIcon(html=plots[i])
    marker.add_child(icon)
    popup = folium.Popup(
        "\Prevalance: {}<br>Sample Period: {}<br>Mean Burden: {}".format(
            round(full_group.infected[i], 3), 
            full_group.date[i],
            round(full_group.mean_burden[i], 3)
        ),
        min_width=120,
        max_width=120
    )
    marker.add_child(popup)
    #map.add_child(marker)
    marker_cluster.add_child(marker)
    i += 1
#map.get_root().add_child(legend)


map.save('map.html')
map